In [1]:
import pandas as pd
import numpy as np
import cobra
from CBModellingFuncs import *
import time
import os
import threadpoolctl

processes = 50
threadpoolctl.threadpool_limits(limits=processes)
os.makedirs("../results/sampling/thinning_test", exist_ok=True)
os.makedirs("../results/dataframes/sampling", exist_ok=True)
cobra_config = cobra.Configuration()
cobra_config.solver = "cplex"

In [2]:
model, m_warnings = cobra.io.validate_sbml_model("../results/iMT1026-NZ.xml")

model.solver = 'cplex'
model.tolerance = 1e-6


rgdict = {}
for g in model.groups:
    for r in g.members:
        
        rgdict[r.id] = g.id.title()

SBML errors in validation, check error log for details.
COBRA errors in validation, check error log for details.
COBRA warnings in validation, check error log for details.


In [3]:
DRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/DerivedRetentInfo.csv", index_col=0, header=[0,1])
IRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/InterpolatedRetentInfo.csv", index_col=0, header=[0,1])
FRinfo = pd.read_csv(f"../results/dataframes/cultivation_data/FittedRetentInfo.csv", index_col=0, header=[0,1])

NGAMseries = pd.read_csv('../results/dataframes/NGAMseries.csv',index_col=0,squeeze=True)

model_viab_cols = ["Growth Rate model 1/h", "qS model mmol/gh","qP model g/gh", "qCO2 viable model mmol/gh", "qO2 viable model mmol/gh", "qStorGlyc mmol/gh"]
viab_cols = ["Average Growth Rate 1/h", "qS viable mmol/gh","qP g/gh", "qCO2 viable mmol/gh", "qO2 viable mmol/gh", "qStorGlyc mmol/gh"] 

## Sampling

In [4]:
co2=True
o2=True

In [5]:
%%time
run_time = []
sampling_results = []

equation_names = ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]
datasets = [DRinfo, DRinfo, IRinfo, FRinfo, DRinfo]

for tfactor in [1,10,100, 1000]:
    for i, equation in enumerate(equation_names):
        data = datasets[i].loc[:, model_viab_cols].copy()
        if equation in ["Consensus", "ScaledConsensus"]:
            data["qStorGlyc mmol/gh"] = 0
        
        time_0 = data.iloc[0,:]
        name = time_0.name
        values = time_0
        mu = values[0]
        sd = values[1]
    
        loop_start = time.time()


        with model as temp_model:
            
            set_biomass_objective(temp_model, equation, name)

            temp_model.reactions.ATPM.bounds = (NGAMseries[f"{equation}"],1000)
           
            solution = constrain_predict(temp_model, values, co2=co2)
           
            pred_mu = solution.objective_value
            
            if equation in ["Consensus", "ScaledConsensus"]:
                temp_model.reactions.get_by_id(equation).bounds = (pred_mu*0.95, pred_mu)
                
            else:
                temp_model.reactions.get_by_id(f"{equation}Biomass{name}").bounds = (pred_mu*0.95, pred_mu)

            flux_span = constrain_predict(temp_model, values, co2=co2, fluxva=True, processes=processes,fopt=0)
            remove_blocked(temp_model, flux_span, solution)


            chains = constrain_predict(temp_model, values, sampling=True, chains=4, thinning=tfactor, 
                                       loopless=True, processes=processes, co2=co2, n_samples=1250)

                 # ensure all have same number of samples, may have
            chains = [df.iloc[:min([df.shape[0] for df in chains]), :] for df in chains]
            print(f"{equation} chains with thinning factor of  {tfactor} all have length of {chains[0].shape[0]}") 

            for j, chain in enumerate(chains):
                chain["Chain"] = j+1
            chains = pd.concat(chains, ignore_index=True)
            chains.to_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz", 
                                compression={'method': 'gzip', 'compresslevel': 6, 'mtime': 1})
            
        loop_end = time.time()
        run_time.append((equation, tfactor, (loop_end - loop_start)/60))
                         
run_time = pd.DataFrame(run_time, columns=["Equation", "Thinning", "Time"])
run_time.to_csv("../results/dataframes/sampling/Thinning_RunTime.csv")

2387 reactions before making consistent, 1425 after
Consensus chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1434 after
Derived chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1442 after
Interpolated chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1441 after
Fitted chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1443 after
ScaledConsensus chains with thinning factor of  1 all have length of 1250
2387 reactions before making consistent, 1439 after
Consensus chains with thinning factor of  10 all have length of 1250
2387 reactions before making consistent, 1440 after
Derived chains with thinning factor of  10 all have length of 1250
2387 reactions before making consistent, 1439 after
Interpolated chains with thinning factor of  10 all have length of 1250
2387 reactions before making con

In [6]:
%%time

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

all_stats = pd.DataFrame()
n_chains = 4

for tfactor in [1,10,100, 1000]:#, 10000]:
    
    con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestConsensus{name}T{tfactor}.pkl.gz")
    con_chains = [con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    scaled_con_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestScaledConsensus{name}T{tfactor}.pkl.gz")
    scaled_con_chains = [scaled_con_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    der_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestDerived{name}T{tfactor}.pkl.gz")
    der_chains = [der_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
   
    int_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestInterpolated{name}T{tfactor}.pkl.gz")
    int_chains = [int_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    fit_chains = pd.read_pickle(f"../results/sampling/thinning_test/TestFitted{name}T{tfactor}.pkl.gz")
    fit_chains = [fit_chains.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
    
    shared_columns = list(set(con_chains[0].columns).intersection(scaled_con_chains[0].columns, 
                                                                  der_chains[0].columns, 
                                                                  int_chains[0].columns, 
                                                                  fit_chains[0].columns))
    
    con_chains = [df.loc[:,shared_columns] for df in con_chains]
    scaled_con_chains = [df.loc[:,shared_columns] for df in scaled_con_chains]
    der_chains = [df.loc[:,shared_columns] for df in der_chains]
    int_chains = [df.loc[:,shared_columns] for df in int_chains]
    fit_chains = [df.loc[:,shared_columns] for df in fit_chains]
 
    all_df = []

    for tup in [("Consensus", con_chains),
                ("ScaledConsensus", scaled_con_chains),
                ("Derived", der_chains),
                ("Interpolated", int_chains),
                ("Fitted", fit_chains)]:
    
        biocomp, model_chains = tup
        grouped, rxns = extractchains(model_chains)
        
        for_pool = list(zip(grouped,rxns))
      
        pool = Pool(processes=processes)
        all_series = pool.starmap(calculateDiagnostics, for_pool)
        pool.close()
        df = pd.concat(all_series,axis=1).T
       
        df["Biomass Composition"] = biocomp
        all_df.append(df)
        
    all_df = pd.concat(all_df, ignore_index=True)
    all_df["Thinning Factor"] = tfactor
    
    all_stats = pd.concat([all_stats, all_df], ignore_index=True)
        
all_stats.set_index(["Thinning Factor", "Biomass Composition"], inplace=True)
all_stats["Rhat Fails"] = all_stats["Rhat"] > 1.01
all_stats["ESS-Bulk Fails"] = all_stats["ESS-Bulk"] < 400
n_grouped_chains = all_stats.groupby(["Thinning Factor", "Biomass Composition"]).count()["Rxn"]


# summary_stats = all_stats[["Geweke Fails","ESS Split Fails", "Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = all_stats[["Geweke Fails","Rhat Fails", "ESS-Bulk Fails"]]
summary_stats = (summary_stats > 0).groupby(["Thinning Factor", "Biomass Composition"]).sum()
temp_df = (summary_stats.divide(n_grouped_chains, axis=0) * 100)
# temp_df.columns=["% Geweke Fails","% seperated-ESS Fails","% Rhat Fails","% bulk-ESS Fails"]
temp_df.columns=["% Geweke Fails","% Rhat Fails","% bulk-ESS Fails"]
summary_stats = pd.concat([summary_stats,temp_df],axis=1)
summary_stats.to_csv(f"../results/dataframes/sampling/Thinning_AllStats.csv")

CPU times: user 14.4 s, sys: 46.9 s, total: 1min 1s
Wall time: 1min 10s


In [7]:
summary_stats

Geweke Fails  Rhat Fails  ESS-Bulk Fails  \
Thinning Factor Biomass Composition                                             
1               Consensus                     829         460             746   
                Derived                       834         399             729   
                Fitted                        754         626             663   
                Interpolated                  844         632             741   
                ScaledConsensus               744         633             662   
10              Consensus                     875         485             608   
                Derived                       900         355             599   
                Fitted                        885         149             671   
                Interpolated                  880         572             582   
                ScaledConsensus               955         597             657   
100             Consensus                     751          27              20   
                Derived                       709           8               8   
                Fitted                        540          36               2   
                Interpolated                  777          74               5   
                ScaledConsensus               777          28              17   
1000            Consensus                     338           3               0   
                Derived                       411           5               0   
                Fitted                        312           8               1   
                Interpolated                  411           3               0   
                ScaledConsensus               648           3              19   

                                     % Geweke Fails  % Rhat Fails  \
Thinning Factor Biomass Composition                                 
1               Consensus                 93.461105     51.860203   
                Derived                   96.751740     46.287703   
                Fitted                    95.081967     78.940731   
                Interpolated              98.253783     73.573923   
                ScaledConsensus           87.017544     74.035088   
10              Consensus                 96.685083     53.591160   
                Derived                   96.359743     38.008565   
                Fitted                    92.962185     15.651261   
                Interpolated              97.023153     63.065050   
                ScaledConsensus           94.181460     58.875740   
100             Consensus                 80.493033      2.893891   
                Derived                   74.789030      0.843882   
                Fitted                    57.385760      3.825717   
                Interpolated              83.458647      7.948443   
                ScaledConsensus           81.789474      2.947368   
1000            Consensus                 36.422414      0.323276   
                Derived                   43.538136      0.529661   
                Fitted                    34.135667      0.875274   
                Interpolated              43.081761      0.314465   
                ScaledConsensus           71.444322      0.330761   

                                     % bulk-ESS Fails  
Thinning Factor Biomass Composition                    
1               Consensus                   84.103720  
                Derived                     84.570766  
                Fitted                      83.606557  
                Interpolated                86.263097  
                ScaledConsensus             77.426901  
10              Consensus                   67.182320  
                Derived                     64.132762  
                Fitted                      70.483193  
                Interpolated                64.167585  
                ScaledConsensus             64.792899  
100             Consensus                    2.143623  
       

In [8]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
roi = ["G6PDH2","PGI","PYK","MDHm","CSm","AKGMALtm","NADH2_u6cm","NADH2_u6mh","CYOR_u6m","CYOOm","ATPS3m","ATPM"]
failures = failures[failures["Rxn"].isin(roi)]

failures.to_csv(f"../results/dataframes/sampling/Thinning_FailuresInterestingReactions.csv")
failures

Rxn Geweke Fails      Rhat  \
Thinning Factor Biomass Composition                                      
1               Consensus                ATPS3m            4  1.039586   
                Consensus                 CYOOm            4   1.02158   
                Consensus                   PGI            1  1.039818   
                Consensus              AKGMALtm            4  1.043052   
                Consensus            NADH2_u6mh            4  1.035989   
...                                         ...          ...       ...   
100             Consensus            NADH2_u6cm            4  1.011055   
                Interpolated                PYK            3  1.012699   
                Interpolated                CSm            3  1.010516   
                Fitted                 AKGMALtm            3  1.013152   
                Fitted                     MDHm            3  1.010487   

                                        ESS-Bulk     ESS-Tail  Rhat Fails  \
Thinning Factor Biomass Composition                                         
1               Consensus             200.119507   221.170529        True   
                Consensus             210.925856   242.117459        True   
                Consensus             186.244444   255.572064        True   
                Consensus             227.631605   221.576272        True   
                Consensus             201.036961   221.681081        True   
...                                          ...          ...         ...   
100             Consensus            1185.142082  2155.617309        True   
                Interpolated          854.295933  1559.075703        True   
                Interpolated          817.383158   1352.95068        True   
                Fitted               1068.471637  1589.037205        True   
                Fitted               1133.406352  1497.911096        True   

                                     ESS-Bulk Fails  
Thinning Factor Biomass Composition                  
1               Consensus                      True  
                Consensus                      True  
                Consensus                      True  
                Consensus                      True  
                Consensus                      True  
...                                             ...  
100             Consensus                     False  
                Interpolated                  False  
                Interpolated                  False  
                Fitted                        False  
                Fitted                        False  

[115 rows x 7 columns]

In [9]:
failures = all_stats[all_stats.loc[:,["Rhat Fails", "ESS-Bulk Fails"]].any(axis=1)].copy()
failures.loc[:,"Subsystem"] = failures["Rxn"].apply(lambda x:rgdict.get(x))

test = failures[(failures["Rhat Fails"] > 0)^(failures["ESS-Bulk Fails"] > 0)]
sum_fails_by_subsystem = test.groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()[["Rhat Fails", "ESS-Bulk Fails"]]
sum_fails_by_subsystem = sum_fails_by_subsystem.groupby(["Thinning Factor", "Subsystem"]).sum().unstack([0])
sum_fails_by_subsystem = sum_fails_by_subsystem.fillna(0)


sum_fails_by_subsystem.to_csv(f"../results/dataframes/sampling/Thinning_FailuresSubsystem.csv")
sum_fails_by_subsystem

Rhat Fails              \
Thinning Factor                                          1     10    100    
Subsystem                                                                   
Alanine And Aspartate Metabolism                          0.0   1.0   3.0   
Amino Sugar And Nucleotide Sugar Metabolism               0.0   0.0   0.0   
Arginine And Proline Metabolism                           0.0   3.0   0.0   
Biomass Composition                                       0.0   0.0   2.0   
Carbohydrate Metabolism                                   0.0   0.0   0.0   
Citric Acid Cycle                                         1.0   3.0   7.0   
Complex Alcohol Metabolism                                0.0   0.0   1.0   
Cysteine Metabolism                                       0.0   2.0   0.0   
Exchange Reaction                                         0.0   0.0   0.0   
Fatty Acid Biosynthesis                                   6.0   3.0  14.0   
Fatty Acid Degradation                                    0.0  27.0   6.0   
Folate Metabolism                                         0.0   0.0   5.0   
Fructose And Mannose Metabolism                           0.0   0.0   0.0   
Galactose Metabolism                                      0.0   0.0   0.0   
Glutamate Metabolism                                      1.0   6.0   2.0   
Glutamine Metabolism                                      0.0   0.0   0.0   
Glycerolipid Metabolism                                   1.0   8.0   2.0   
Glycine And Serine Metabolism                             0.0   0.0   5.0   
Glycolysis/Gluconeogenesis                                0.0   0.0   6.0   
Glycosylation Pathway                                     0.0   0.0   0.0   
Glyoxylate And Dicarboxylate Metabolism                   3.0   1.0   1.0   
Histidine Metabolism                                      0.0   3.0   0.0   
Inositol Phosphate Metabolism                             0.0   0.0   0.0   
Methanol Metabolism                                       0.0   3.0   1.0   
Methionine Metabolism                                     2.0   8.0   0.0   
N-Glycan Metabolism                                       0.0   0.0   2.0   
Nad Biosynthesis                                          3.0   0.0   3.0   
Nitrogen Metabolism                                       0.0   2.0   0.0   
Nucleotide Metabolism                                     3.0  20.0   9.0   
Other                                                     0.0   0.0   1.0   
Other Amino Acid Metabolism                               1.0   1.0   6.0   
Oxidative Phosphorylation                                 1.0   3.0   1.0   
Pantothenate And Coa Biosynthesis                         1.0   1.0   7.0   
Pentose Phosphate Pathway                                 0.0   1.0   0.0   
Phospholipid Metabolism                                   0.0  15.0   3.0   
Porphyrin And Chlorophyll Metabolism                      0.0   0.0   1.0   
Pyridoxine Metabolism                                     5.0  16.0   5.0   
Pyruvate Metabolism                                       5.0   9.0   4.0   
Quinone Biosynthesis                                      0.0   0.0   0.0   
Recombinant Protein Production                            0.0   0.0   0.0   
Riboflavin Metabolism                                     0.0   7.0   6.0   
Sphingolipid Metabolism                                   3.0  35.0   0.0   
Sterol Metabolism                                         1.0   1.0   1.0   
Thiamine Metabolism                                       7.0   3.0   0.0   
Threonine And Lysine Metabolism                           0.0   2.0   0.0   
Transport, Er                                             0.0   6.0   5.0   
Transport, Extracellular                                  0.0   0.0   0.0   
Transport, Golgi Apparatus                                0.0   6.0   0.0   
Transport, Mitochondrial                                  4.0   3.0  11.0   
Transport, Nuclear                             

In [10]:
rhat_fails = failures[failures["Rhat Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
rhat_fails = rhat_fails.groupby(["Thinning Factor", "Biomass Composition"])["Rhat Fails"].nlargest(5)
rhat_fails.index = rhat_fails.index.droplevel([0,1])
rhat_fails = rhat_fails.to_frame()

rhat_fails.to_csv(f"../results/dataframes/sampling/Thinning_FailuresRhat.csv")
rhat_fails

Rhat Fails
Thinning Factor Biomass Composition Subsystem                                             
1               Consensus           Fatty Acid Biosynthesis                             60
                                    Transport, Mitochondrial                            42
                                    Nucleotide Metabolism                               26
                                    Transport, Peroxisomal                              20
                                    Valine, Leucine, And Isoleucine Metabolism          19
...                                                                                    ...
1000            Fitted              Transport, Extracellular                             1
                Interpolated        Exchange Reaction                                    1
                                    Glycerolipid Metabolism                              1
                                    Transport, Extracellular                             1
                ScaledConsensus     Nucleotide Metabolism                                3

[90 rows x 1 columns]

In [11]:
essbulk_fails = failures[failures["ESS-Bulk Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).sum()
if not essbulk_fails.empty:
    sum_essbulk = essbulk_fails.groupby(["Thinning Factor", "Biomass Composition"])["ESS-Bulk Fails"].nlargest(5)
    sum_essbulk.index = sum_essbulk.index.droplevel([0,1])
    sum_essbulk = sum_essbulk.to_frame()
    sum_essbulk.to_csv(f"../results/dataframes/sampling/Thinning_FailuresSumESS.csv")
    
else:
    
    sum_essbulk = "No ESS Bulk fails"

sum_essbulk

                                                              ESS-Bulk Fails
Thinning Factor Biomass Composition Subsystem                               
1               Consensus           Fatty Acid Biosynthesis               77
                                    Transport, Mitochondrial              58
                                    Sphingolipid Metabolism               56
                                    Fatty Acid Degradation                54
                                    Nucleotide Metabolism                 47
...                                                                      ...
1000            ScaledConsensus     Nucleotide Metabolism                  6
                                    Quinone Biosynthesis                   6
                                    Histidine Metabolism                   3
                                    Nitrogen Metabolism                    2
                                    Transport, Mitochondrial               2

In [12]:
geweke_fails = failures[failures["Geweke Fails"] > 0].groupby(["Thinning Factor", "Biomass Composition", "Subsystem"]).count()
geweke_fails = geweke_fails.groupby(["Thinning Factor", "Biomass Composition"])["Geweke Fails"].nlargest(5)
geweke_fails.index = geweke_fails.index.droplevel([0,1])
geweke_fails = geweke_fails.to_frame()


geweke_fails.to_csv(f"../results/dataframes/sampling/Thinning_FailuresGeweke.csv")
geweke_fails

Geweke Fails
Thinning Factor Biomass Composition Rxn                    
1               Consensus           10FTHFtm              1
                                    12DGRter              1
                                    13GS                  1
                                    1PYR5Ctm              1
                                    25DAPTPM              1
...                                                     ...
1000            ScaledConsensus     34HPLFM               1
                                    34HPPt2m              1
                                    4HBZCOAFm             1
                                    4HBZFm                1
                                    4HBZtm                1

[94 rows x 1 columns]

### Additional diagnostic plots of a selection of reactions (used in later analysis)
Additional plots include Geweke plots, Rank plots, local, quantile and evolution ESS plots

In [13]:
%%time

from matplotlib.backends.backend_pdf import PdfPages

data = DRinfo.loc[:,model_viab_cols]

time_0 = data.iloc[0,:]
name = time_0.name
values = time_0
mu = values[0]

n_chains = 4
interesting_reactions = ["G6PDH2","PGI","PC","PYK","MDHm", 
                         "CSm", "AKGMALtm","NADH2_u6cm","NADH2_u6mh",
                         "SUCD2_u6m","CYOR_u6m","CYOOm","ATPS3m"]

for rxn in interesting_reactions:

    with PdfPages(f'../results/plots/SamplingDiagnostics{rxn}.pdf') as pdf:
        
        for equation in ["Consensus", "Derived", "Interpolated", "Fitted","ScaledConsensus"]:
            
            fig = plt.figure(figsize=(20, 18), constrained_layout=True)
            spec = fig.add_gridspec(nrows=4, ncols=5, height_ratios=[1,1,1,1], hspace=0.1, bottom=0.1)
            
            for i, tfactor in enumerate([1,10,100, 1000]):
                
                chains_df = pd.read_pickle(f"../results/sampling/thinning_test/Test{equation}{name}T{tfactor}.pkl.gz")
                chains_df = [chains_df.set_index("Chain").loc[j+1].reset_index(drop=True) for j in range(n_chains)]
                
                grouped, rxns = extractchains(chains_df)
                
                chains = grouped[rxns.index(rxn)]
                cmap = plt.cm.get_cmap('tab10', 10)
                color_list = [matplotlib.colors.rgb2hex(cmap(j)[:3]) for j in range(cmap.N)][:4]

                axes = [fig.add_subplot(spec[i, j]) for j in np.arange(5)]

                plottrace(chains,axes[0],color_list)
                axes[0].set_ylabel("Trace Plot",fontsize=15)
                
                plotrank(chains,axes[1],color_list)
                axes[1].set_ylabel("Rank Plot",fontsize=15)
                
                az.plot_ess(chains,ax=axes[2], relative=True, kind="local")
                az.plot_ess(chains,ax=axes[3], relative=True, kind="quantile")
                az.plot_ess(chains,ax=axes[4], relative=True, kind="evolution")
                
                [axes[i].set_ylim(0,1.4) for i in [2,3,4]]
                [axes[i].set_title("") for i in [1,2,3,4]]
                [ax.tick_params(labelsize=15) for ax in axes]

                axes[2].set_title(f"{rxn} - {biocomp} - Thinning factor {tfactor}",fontsize=20, pad=30)

            pdf.savefig(fig)
            plt.close()
                    
               

CPU times: user 3min 28s, sys: 11.3 s, total: 3min 39s
Wall time: 3min 40s
